<h1>PAGERANK</h1>

In [11]:
import numpy as np
import pandas as pd

In [16]:
data_set = pd.read_csv('../data/soc-sign-bitcoinotc.csv')
data_set.columns = ['source', 'target', 'rating', 'time']
data_set.head()

,source,target,rating,time
0,6,5,2,1.289242e+09
1,1,15,1,1.289243e+09
2,4,3,7,1.289245e+09
3,13,16,8,1.289254e+09
4,13,10,8,1.289254e+09


In [15]:
'''
    Filtring data
        getting only rating >= 8
'''
data_set = data_set.query('rating >= 8')


In [23]:
def gen_adjacence_dict(data):
    adj_dict = dict()
    
    data_size = data_set.shape[0]
    
    for index in range(data_size):
        source = data.loc[index, 'source']
        target = data.loc[index, 'target']
        add_to_adj(adj_dict, source, target)
        
    return adj_dict

def add_to_adj(adj, source, target):
    if (source in adj):
        adj[source].append(target)
    else:
        adj[source] = [target]
print(len(gen_adjacence_dict(data_set)))

4814


In [ ]:
for i in range(data_set.shape[0]):
    